In [9]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q -U bitsandbytes
!pip install -q accelerate
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
meetingbank = load_dataset("huuuyeah/meetingbank")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
train_data = meetingbank['train']
print(train_data)

Dataset({
    features: ['summary', 'uid', 'id', 'transcript'],
    num_rows: 5169
})


In [4]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.summarize import load_summarize_chain

In [5]:
dataset_name = "huuuyeah/meetingbank"
page_content_column = "transcript"
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

In [6]:
data = loader.load()
data[:10]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


[Document(page_content='"Please refrain from profane or obscene speech. Direct your comments to council as a whole and refrain from individual or personal attacks. Councilwoman Gilmore, will you please put Council Bill 161 on the floor? Yes, President Brooks, I move that council bill 161 as amended, be placed upon final consideration and do pass. It has been moved and seconded. Councilman. Clerk, go ahead and offer your motion to further amend. Thank you, Mr. President. I move that council bill 161 as amended be further amended in the following particulars on page five, line one, strike 13 .1.9 and replace with 13 .1. ten on page five. Line three strike 13 1.9.3 and replace with 13 one dot and four. Add a new section to the bill which reads as follows Section six A except as otherwise provided in Section six B of this ordinance. With respect to certain site development plan applications, the amendments to the Denver Zoning Code adopted by this ordinance take effect on May five, 2017. B

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()
docs = splitter.split_documents(data[:1])

In [ ]:
import torch

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    # quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 50
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(
    pipeline=pipeline,
    )


In [ ]:
from langchain.chains.summarize import load_summarize_chain

prompt_template = """[INST]
Act as a professional technical meeting minutes writer.
Tone: formal
Format: Technical meeting summary
Length:  200 ~ 300
Use bullet points if needed

{text}

CONCISE SUMMARY IN ITALIAN:
[/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

print(chain.run(docs[12:13]))